<a href="https://colab.research.google.com/github/conrizzo/conradswebsite/blob/master/Transformer_SBERT_project_August1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers
!pip install keybert

from sentence_transformers import SentenceTransformer
# paraphrase-mpnet-base-v2
# paraphrase-MiniLM-L6-v2
model = SentenceTransformer('paraphrase-mpnet-base-v2') 
print(model)


# Test keyBERT

from keybert import KeyBERT

kw_model = KeyBERT(model='paraphrase-mpnet-base-v2')


     |████████████████████████████████| 85 kB 1.4 MB/s 
     |████████████████████████████████| 2.6 MB 12.9 MB/s 
     |████████████████████████████████| 1.2 MB 63.8 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 3.3 MB 62.0 MB/s 
     |████████████████████████████████| 895 kB 59.5 MB/s 
     |████████████████████████████████| 636 kB 71.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=2ae26054fa497b36ce07c3589a37776e24e5846d154b67b8daeeeb8ab93debc7
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 208 kB 10.1 MB/s 
     |████████████████████████████████| 51 kB 7.9 MB/s 
  Crea


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


Here is where I am copying the database I made - which is another database with my own entries added.

In [2]:
# Import package
import urllib
from urllib.request import urlretrieve


# https://arxiv.org/abs/2006.09479   Idiom expression dataset

# Assign url of file: url
#  https://raw.githubusercontent.com/prateeksaxena2809/EPIE_Corpus/master/Formal_Idioms_Corpus/Formal_Idioms_Candidates.txt


# Thank you for this idiom data base to start
#url = "https://raw.githubusercontent.com/dice-group/LIdioms/master/en/english.ttl"

url = "https://raw.githubusercontent.com/conrizzo/english_idiom_database/main/semantic_relations_as_TSV"

#url = "https://github.com/prateeksaxena2809/EPIE_Corpus/blob/master/Formal_Idioms_Corpus/Formal_Idioms_Candidates.txt"
file = urllib.request.urlopen(url)

#rdfs:label
#skos:definition

idiom_list = []

# https://aclanthology.org/L18-1392.pdf

import re

# Extract all quoted text
for line in file:
  decoded_line = line.decode("utf-8")
  idiom_list.append(decoded_line.strip())    

idiom_corpus = []
idiom_semantic_meaning = []
formatted_idiom_values = []

for i in range(len(idiom_list)):
  idiom_corpus.append(idiom_list[i].split('    ')) 
  


#for i in idiom_corpus:
 # print(i[1])
  #print(i[0],'  ', i[1])
idiom,semantic = zip(*idiom_corpus)

print("Number of idioms: ", len(idiom))






print(semantic)


Number of idioms:  304
('a quick temper', 'someone that can do many different jobs.', 'a wild guess; an attempt to do something when there is little chance of success', ' completely certain that something is true or will happen', "people's intentions can be judged better by what they do than what they say.", 'make a bad situation even worse', 'meaningless and incomprehensible', 'when everyone is facing the same challenges', 'very expensive; a large amount of money', 'willing to do something immediately, on the slightest signal or urging', 'having to start all over again', "when an attempt fails and it's time to start all over", 'be happy when a person leaves.', 'to force an issue that has already ended', 'avoiding the main topic', 'to serve anyone or fulfill any wish.', 'old-fashioned', 'absolutely sure about something or someone', 'to take on a task that is way too big.', 'to stop yourself from saying something that you would really like to say', "something good that isn't recognized 

In [3]:

# This is interesting. Running all the semantic meanings of idioms through keyBERT to find words that most represent the combined semantic meaning of the definitions emphasizes how it's hard to know what an idiom means.

# So the plan here is, we have semantic definitions of the idioms. THen we use keyBERT to extract the most different (weighted high by diversity) key words to create a uniform semantic definition for every idiom.


# run semantic meanings through keyword extraction
doc_list = semantic

doc = """

to make a situation more confused and less easy to understand or deal with

  
"""
#kw_model = KeyBERT()

count = 0

keybert_semantic_extraction = []

for i in doc_list:
    print(count)
    count+=1
  
    #print(i)
    keywords = kw_model.extract_keywords(doc)
    try:
      #highlight=True
      multi_word_keys = kw_model.extract_keywords(i, keyphrase_ngram_range=(1,5), stop_words='english', 
                                    use_mmr=True, diversity=0.8, nr_candidates=1, top_n=1)
      
      #print(multi_word_keys)
      if multi_word_keys == []:
        keybert_semantic_extraction.append(i)
      else:
        keybert_semantic_extraction.append(multi_word_keys)
    except ValueError:
      ("ValueError: max() arg is an empty sequence")
    #if multi_word_keys != []:
    
    print(multi_word_keys)
print(keybert_semantic_extraction)
print(len(keybert_semantic_extraction))

0
[('quick temper', 0.9567)]
1
[('different jobs', 0.6444)]
2
[('wild guess attempt little chance', 0.7877)]
3
[('completely certain true happen', 0.8186)]
4
[('people intentions judged better', 0.7571)]
5
[('make bad situation worse', 0.9504)]
6
[('meaningless incomprehensible', 0.945)]
7
[('facing challenges', 0.7014)]
8
[('expensive large money', 0.9011)]
9
[('willing immediately slightest signal urging', 0.8135)]
10
[('having start', 0.5143)]
11
[('attempt', 0.5846)]
12
[('happy person leaves', 0.8904)]
13
[('force issue ended', 0.5605)]
14
[('avoiding main topic', 0.9689)]
15
[('serve fulfill wish', 0.6998)]
16
[('old fashioned', 0.9796)]
17
[('absolutely sure', 0.7021)]
18
[('task way big', 0.8529)]
19
[('stop saying really like say', 0.6193)]
20
[('good isn recognized', 0.4932)]
21
[('stupid person', 0.959)]
22
[('extremely unhappy interesting', 0.6555)]
23
[('extremely cold weather', 1.0)]
24
[('break initial awkwardness start conversation', 0.949)]
25
[('nervous', 0.8272)]
26


In [6]:
    #print(multi_word_keys)
print(keybert_semantic_extraction)
print(len(keybert_semantic_extraction))

count = 0
#for r in keybert_semantic_extraction:
  # if the mmr similarity is greater than 0.8 we can say the keywords might give a better semantic representation than a longer definition for idiom matching
  #print(zz)
#if r[0][1] > 0.6:
#       print(count)
#       print(r)
 #      count+=1
  
  

[[('quick temper', 0.9567)], [('different jobs', 0.6444)], [('wild guess attempt little chance', 0.7877)], [('completely certain true happen', 0.8186)], [('people intentions judged better', 0.7571)], [('make bad situation worse', 0.9504)], [('meaningless incomprehensible', 0.945)], [('facing challenges', 0.7014)], [('expensive large money', 0.9011)], [('willing immediately slightest signal urging', 0.8135)], [('having start', 0.5143)], [('attempt', 0.5846)], [('happy person leaves', 0.8904)], [('force issue ended', 0.5605)], [('avoiding main topic', 0.9689)], [('serve fulfill wish', 0.6998)], [('old fashioned', 0.9796)], [('absolutely sure', 0.7021)], [('task way big', 0.8529)], [('stop saying really like say', 0.6193)], [('good isn recognized', 0.4932)], [('stupid person', 0.959)], [('extremely unhappy interesting', 0.6555)], [('extremely cold weather', 1.0)], [('break initial awkwardness start conversation', 0.949)], [('nervous', 0.8272)], [('buy car constantly gives problems', 0.856

In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)
 
 
import nltk

Wed Aug  4 10:20:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    36W / 250W |   1399MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# THIS ALL FUNCTIONS better if the input sentence is not ambigious with a negation. Try not to use ambigious negations or unconcise sentences if possible.

In [9]:
# NLTK RESOURCES TO TOKENIZE SENTENCES FOR TESTING THE IDIOMS AT A LARGE SCALE


!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

import pandas as pd
import matplotlib.pyplot as plt
import io
import unicodedata
import numpy as np
import re
import string



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [15]:
"""
This is a simple application for sentence embeddings: semantic search
 
We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.
 
This script outputs for various queries the top 5 most similar sentences in the corpus.
""" 
# Research process ----
 
#  https://github.com/prateeksaxena2809/EPIE Corpus
 
from sentence_transformers import SentenceTransformer, util
import torch

# Nearly all the pre-trained models seem to work well, but should test results across a few.
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')
 
 
# -------------------------- NLTK tokenize
from nltk.tokenize import sent_tokenize

test_text =  """something that will never ever happen. The best player on the team is Michael Jackson. The stuff he says is nonsense. He does everything incorrectly. The work is not good enough. He does make good coffee though. I like pizza.   """
test_document = sent_tokenize(test_text)


keywords = kw_model.extract_keywords(test_text)


  #highlight=True
multi_word_keys = kw_model.extract_keywords(test_text, keyphrase_ngram_range=(2,8), stop_words='english', 
                                use_mmr=True, diversity=0.8, nr_candidates=3, top_n=3)
  


print(multi_word_keys)
test_document = multi_word_keys
corpus = semantic

  
corpus_embeddings = embedder.encode(corpus, batch_size = 10, convert_to_tensor=True)
 
# Query sentences individually:







#### ENTER TEXT QUERIE VALUE HERE #####
queries = ["something that will never ever happen", "dogs have a good day", "I never seen so much water around here!"]









#queries = test_document
# we can run a test document, or individually enter queries

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
 
    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
 
    print("\n\n======================\n\n")
    print("Query:", query)
    #print("\nTop 5 most similar sentences in corpus:")
 
    # Make a tuple of 3 values
    # Tuple(The definiton match, the semantic relatedness, the location in the corpus for tagging it to idioms)
 
    for score, idx in zip(top_results[0], top_results[1]):
        #print(idx)
        #print(score)
        if score > 0.25:
          print("Idiom: ", idiom[idx], semantic.index(semantic[idx]), "(Score: {:.4f})".format(score))
          #print("Idiom: ", idiom[idx], "Semantic relation: ", semantic[idx], semantic.index(semantic[idx]), "(Score: {:.4f})".format(score))
        
        # To reverse lookup values with a dictionary ---


    


[('best player team michael jackson stuff says nonsense', 0.5825), ('good coffee like pizza', 0.3577), ('work good does', 0.18)]




Query: something that will never ever happen
Idiom:  when pigs fly 121 (Score: 1.0000)
Idiom:  a cold day in hell 242 (Score: 0.8438)
Idiom:  once in a blue moon 87 (Score: 0.6396)
Idiom:  wouldn't be caught dead 122 (Score: 0.6116)
Idiom:  i'll eat my hat if... 283 (Score: 0.5778)




Query: dogs have a good day
Idiom:  the icing on the cake 237 (Score: 0.3175)
Idiom:  crème de la crème 224 (Score: 0.3061)
Idiom:  on cloud nine 140 (Score: 0.2614)
Idiom:  sleep on it 104 (Score: 0.2582)
Idiom:  be in seventh heaven 146 (Score: 0.2529)




Query: I never seen so much water around here!
Idiom:  its raining cats and dogs 176 (Score: 0.4134)
Idiom:  a stone's throw 270 (Score: 0.2784)
Idiom:  know like the back of your hand ? 180 (Score: 0.2511)


In [11]:
from sentence_transformers import SentenceTransformer, util


query_embedding = model.encode('All hat and no cattle')
passage_embedding = model.encode('They seem impressive or exciting but in reality they do not do much')

print("Similarity:", util.pytorch_cos_sim(query_embedding, passage_embedding))

Similarity: tensor([[0.0614]])


In [12]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
text = "He let the cat out of the bag."
encoded_input = tokenizer(text, return_tensors='pt')

In [13]:
# code for k means clusters for analysis

print("end")
"""  from sklearn.cluster import KMeans        

# Perform kmean clustering
num_clusters = 75
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append((idiom[sentence_id]))
#corpus[sentence_id], 
for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster) """

end


'  from sklearn.cluster import KMeans        \n\n# Perform kmean clustering\nnum_clusters = 75\nclustering_model = KMeans(n_clusters=num_clusters)\nclustering_model.fit(corpus_embeddings)\ncluster_assignment = clustering_model.labels_\n\nclustered_sentences = [[] for i in range(num_clusters)]\nfor sentence_id, cluster_id in enumerate(cluster_assignment):\n    clustered_sentences[cluster_id].append((idiom[sentence_id]))\n#corpus[sentence_id], \nfor i, cluster in enumerate(clustered_sentences):\n    print("Cluster ", i+1)\n    print(cluster) '